In [46]:
import cv2 
import mediapipe as mp
import pyautogui
import numpy as np

In [47]:
# Initialize hand detection 
mp_hands = mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5, model_complexity=1, static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

pyautogui.FAILSAFE = False
click_threshold = 0.1
alpha = 0.5

# define hand landmark to track
wrist_landmark_id = mp.solutions.hands.HandLandmark.WRIST
index_landmark_id = mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP
thumb_landmark_id = mp.solutions.hands.HandLandmark.THUMB_TIP
middle_landmark_id = mp.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP

def is_left_click(landmarks):
    index_tip = landmarks.landmark[index_landmark_id]
    thumb_tip = landmarks.landmark[thumb_landmark_id]
    distance = np.sqrt((index_tip.x - thumb_tip.x) ** 2 + (index_tip.y - thumb_tip.y) ** 2)
    return distance <= click_threshold

def is_right_click(landmarks):
    middle_tip = landmarks.landmark[middle_landmark_id]
    thumb_tip = landmarks.landmark[thumb_landmark_id]
    distance = np.sqrt((middle_tip.x - thumb_tip.x) ** 2 + (middle_tip.y - thumb_tip.y) ** 2)
    return distance <= click_threshold

def get_smoothed_landmarks(landmarks):
    """
    Calculates smoothed wrist and index tip coordinates using exponential smoothing,
    handling cases where no hands are detected.

    Args:
        landmarks: MediaPipe hand landmarks object.

    Returns:
        A tuple containing the smoothed wrist and index tip coordinates (x, y) or None if no hands are detected.
    """
    
    global wrist_prev, index_tip_prev
    if not landmarks:
        # No hands detected, return None to indicate this
        return None

    wrist = landmarks.landmark[mp.solutions.hands.HandLandmark.WRIST]
    index_tip = landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]

    # Initialize on the first frame
    if wrist_prev is None:
        wrist_prev = wrist
    if index_tip_prev is None:
        index_tip_prev = index_tip

    wrist_smoothed = alpha * wrist.x + (1 - alpha) * wrist_prev.x, alpha * wrist.y + (1 - alpha) * wrist_prev.y
    index_tip_smoothed = alpha * index_tip.x + (1 - alpha) * index_tip_prev.x, alpha * index_tip.y + (1 - alpha) * index_tip_prev.y

    wrist_prev = wrist
    index_tip_prev = index_tip

    return wrist_smoothed, index_tip_smoothed


def get_cursor_movement(landmarks):
    try:
        wrist_smoothed, index_tip_smoothed = get_smoothed_landmarks(landmarks)
    except ValueError as e:
        # print(e)
        return 0, 0

    movement_x = (wrist_smoothed[0] - index_tip_smoothed[0]) * (screen_width / 800)
    movement_y = (wrist_smoothed[1] - index_tip_smoothed[1]) * (screen_height/600)

def rotate_screen(landmarks):
    index_tip = landmarks.landmark[index_landmark_id]
    print(f"x: {(index_tip.y - index_tip.x)}")
    return abs(index_tip.x - index_tip.y) < 0.01


# get screen size
screen_width, screen_height = pyautogui.size()

# sensitivity factor for cursor movement (adjust for desired speed)
sensivity = 0.5



# Initialize previous landmark positions
prev_fingertip_x = 0
prev_fingertip_y = 0




    
    
    
    


In [48]:

        
def detect_hands(frame):
  """
  Detects hands in a frame using MediaPipe.

  Args:
      frame: The frame to be processed.

  Returns:
      A list of hand landmarks if a hand is detected, otherwise None.
  """
  rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for MediaPipe
  results = mp_hands.process(rgb_frame)
  if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]  # Assuming only one hand
    return hand_landmarks
  else:
    return None
  
wrist_prev = None
index_tip_prev = None
thumb_tip_prev = None

while True:
  # Take a picture with the camera
  success, frame = cap.read()

  # 2. Spot the Tips (continued) - Find the fingertip in this picture
  hand_landmarks = detect_hands(frame)  # This function is defined later

  # If we found a fingertip, let's move the cursor!
  if hand_landmarks:
    # Remember where the fingertip is now
    fingertip_landmark = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
    mp_drawing.draw_landmarks(frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
    fingertip_x = int(fingertip_landmark.x * screen_width)
    fingertip_y = int(fingertip_landmark.y * screen_height)
    
    pyautogui.moveTo(fingertip_x, fingertip_y)

    
    

    # Remember where the fingertip is for next time
    # prev_fingertip_x = smoothed_movement_x
    # prev_fingertip_y = smoothed_movement_y
    
    # if is_left_click(hand_landmarks):
    #     pyautogui.click(button='left')
        
    # if is_right_click(hand_landmarks):
    #     pyautogui.click(button='right')
        
    if rotate_screen(hand_landmarks):
        pyautogui.hotkey('ctrl', 'alt', 'right')

  # Show the picture with the fingertip (optional)
  cv2.imshow('Hand', frame)

  # Close the program if you press 'Esc'
  if cv2.waitKey(5) & 0xFF == 27:
    break

# Clean up the camera
cap.release()
cv2.destroyAllWindows()



c:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


x: 0.04435434937477112
x: -0.022196322679519653
x: -0.09775221347808838
x: -0.11166355013847351
x: -0.11527419090270996
x: -0.18272334337234497
x: -0.16137397289276123
x: -0.19317567348480225
x: -0.15170609951019287
x: -0.12476891279220581
x: -0.0801163911819458
x: -0.04003608226776123
x: -0.03655344247817993
x: 0.004166752099990845
x: 0.05606696009635925
x: 0.06339949369430542
x: 0.07183924317359924
x: 0.07709145545959473
x: 0.08482420444488525
x: 0.09864082932472229
x: 0.10120585560798645
x: 0.1034386157989502
x: 0.10915890336036682
x: 0.10955315828323364
x: 0.11075645685195923
x: 0.11262232065200806
x: 0.11563742160797119
x: 0.11685732007026672
x: 0.11481702327728271
x: 0.11767652630805969
x: 0.11964145302772522
x: 0.11893212795257568
x: 0.11855754256248474
x: 0.11772769689559937
x: 0.11800575256347656
x: 0.1188536286354065
x: 0.12203124165534973
x: 0.12239956855773926
x: 0.12483757734298706
x: 0.13467687368392944
x: 0.2564060389995575
x: 0.42817145586013794
x: 0.47894343733787537
x